In [1]:
import pandas as pd
import numpy as np
#
import faiss
import pyterrier as pt

In [2]:
if not pt.started():
    pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
# Intersección
def list_intersection(l1, l2):
    s1 = set(l1.tolist()[0])
    s2 = set(l2.tolist()[0])
    return len(s1.intersection(s2)) / len(s1)

---
### MAIN
---

In [4]:
dataset = pt.get_dataset("vaswani")
print("Corpus Vaswani: %s " % dataset.get_corpus())

Corpus Vaswani: ['/home/jovyan/.pyterrier/corpora/vaswani/corpus/doc-text.trec'] 


In [5]:
documents = pd.DataFrame(dataset.get_corpus_iter())
documents.shape

(11429, 2)

In [6]:
documents.head()

,docno,text
0,1,compact memories have flexible capacities a d...
1,2,an electronic analogue computer for solving sy...
2,3,electronic coordinate transformer circuit det...
3,4,the british computer society report of a conf...
4,5,millimicrosecond digital computer logic a sys...


In [49]:
#!pip install transformers
#!pip install sentence-transformers
#from sentence_transformers import SentenceTransformer, util
#model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

#doc_text = list(documents['text'].head(10))
#demo_embeddings = model.encode(doc_text, convert_to_tensor=True)
#print (demo_embeddings.shape)
#
#demo_embeddings

In [11]:
# Cargo los embeddings de todos los docs previamente calculados.
vaswani_docs_embeddings = np.load("../data/vaswani_docs_embeddings-512.npy")
vaswani_docs_embeddings.shape

(11429, 512)

In [48]:
vaswani_docs_embeddings[1]

array([ 9.95724797e-02,  7.84335379e-03, -4.75690439e-02, -3.09009496e-02,
        9.55666378e-02, -3.18422914e-02,  6.99837357e-02, -1.73974298e-02,
        1.46257672e-02,  1.28097497e-02,  4.78746806e-04, -2.91474462e-02,
       -1.60487369e-02, -3.17740301e-03,  6.00390509e-02,  5.03449924e-02,
        7.54538029e-02, -2.97647063e-02, -6.68815337e-03, -2.77290437e-02,
        1.18169188e-02, -4.04140614e-02,  2.56132428e-03, -2.95873974e-02,
        6.13711588e-02,  3.66782546e-02, -3.41790654e-02,  1.26804560e-02,
       -1.11371893e-02,  2.62570418e-02, -3.00066695e-02,  2.83214115e-02,
        1.01238757e-01,  1.24692842e-02,  1.41052268e-02,  3.02290544e-02,
        3.11574270e-03, -5.15380800e-02,  1.17427520e-02, -1.47100817e-02,
        7.09908009e-02, -2.94207297e-02,  7.34805455e-03, -6.60830215e-02,
        9.61912982e-03,  3.35156433e-02,  3.22302282e-02,  1.16466114e-03,
        3.97732966e-02,  7.16824876e-03,  2.03587748e-02,  5.89668490e-02,
        9.74677950e-02,  

In [12]:
# Obtengo los 'topics' asociados al corpus
topics = dataset.get_topics()
topics.head()

,qid,query
0,1,measurement of dielectric constant of liquids ...
1,2,mathematical analysis and design details of wa...
2,3,use of digital computers in the design of band...
3,4,systems of data coding for information transfer
4,5,use of programs in engineering testing of comp...


In [14]:
# Cargo los embeddings de todos los queries previamente calculados.
vaswani_query_embeddings = np.load("../data/vaswani_query_embeddings-512.npy")
vaswani_query_embeddings.shape

(93, 512)

### Indexación con FAISS (diferentes índices)
**Más sobre los tipos de índices en FAISS:** https://github.com/facebookresearch/faiss/wiki/Faiss-indexes

**Prueba 1 - Flat Index**  
Recordar: En este tipo de índice se mide la distancia L2 (euclídea) entre el vector de query 
y todos los vectores de documentos almacenados. Es simple y preciso (pero no demasiado rápido).

In [15]:
# Inicialización
d = 512
indexFlat = faiss.IndexFlatL2(d)

# Chequeo cantidad de docs en el índice
indexFlat.ntotal

0

In [16]:
# Agrego los documentos al índice
%time
indexFlat.add(vaswani_docs_embeddings)
indexFlat.ntotal

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


11429

In [17]:
indexFlat.is_trained

True

In [21]:
# Ejemplo de recuperación
k = 4
query_vector = np.array([vaswani_query_embeddings[0]])

In [22]:
%time
DFlat, rsFlat = indexFlat.search(query_vector, k)  # Búsqueda

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


In [23]:
print(rsFlat)

[[1501 7125 1087 1896]]


In [24]:
DFlat

array([[0.49462646, 0.76442146, 0.8314605 , 0.8432368 ]], dtype=float32)

In [25]:
faiss.write_index(indexFlat, "vaswani_faiss_flat.ndx")

**Prueba 2 - IVF Flat Index**  
Recordar: En este tipo de índice se particiona el espacio de búsqueda (nlist) para realizar
un ANN.

In [26]:
nlist = 50  # Cantidad de celdas
quantizer = faiss.IndexFlatL2(d)
indexIVFFlat = faiss.IndexIVFFlat(quantizer, d, nlist)

In [27]:
indexIVFFlat.is_trained

False

In [28]:
# Preparo (train) las estructuras de datos del índice
%time
indexIVFFlat.train(vaswani_docs_embeddings)
indexIVFFlat.ntotal

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


0

In [29]:
indexIVFFlat.is_trained

True

In [30]:
# Agrego los documentos al índice
indexIVFFlat.add(vaswani_docs_embeddings)
indexIVFFlat.ntotal

11429

In [31]:
# Ejemplo de recuperación
k = 5
query_vector = np.array([vaswani_query_embeddings[0]])

In [32]:
%time
DIVFFlat, rsIVFFlat = indexIVFFlat.search(query_vector, k)  # Búsqueda

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [33]:
rsIVFFlat

array([[8275, 4255, 8581, 8481,   57]])

In [34]:
list_intersection(rsFlat, rsIVFFlat)

0.0

In [35]:
indexIVFFlat.nprobe = 10 # Aumentamos el ámbito de búsqueda a 10 celdas
DIVFFlat, rsIVFFlat = indexIVFFlat.search(query_vector, k)  # Búsqueda

In [36]:
rsIVFFlat

array([[1501, 7125, 1896, 1987, 4816]])

In [37]:
faiss.write_index(indexIVFFlat, "vaswani_faiss_ivfflat.ndx")

**Prueba 3 - IVF Flat Index con Product Quantization**  
Recordar: En este tipo de índice se aplica Product Quantization para reducir (comprimir) los vectores. 
Luego se ejecuta ANN.

In [38]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
indexIVFPQ = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

In [39]:
indexIVFPQ.is_trained

False

In [40]:
# Preparo (train) las estructuras de datos del índice
%time
indexIVFPQ.train(vaswani_docs_embeddings)
indexIVFPQ.ntotal

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.25 µs


0

In [41]:
# Agrego los documentos al índice
indexIVFPQ.add(vaswani_docs_embeddings)
indexIVFPQ.ntotal

11429

In [42]:
# Ejemplo de recuperación
k = 5
query_vector = np.array([vaswani_query_embeddings[0]])

In [43]:
%time
DIVFPQ, rsIVFPQ = indexIVFPQ.search(query_vector, k)  # Búsqueda

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [44]:
rsIVFPQ

array([[1519, 8275, 3326, 4255, 8481]])

In [45]:
indexIVFPQ.nprobe = 10
%time
DIVFPQ, rsIVFPQ = indexIVFPQ.search(query_vector, k)  # Búsqueda

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs


In [46]:
rsIVFPQ

array([[1501, 7125, 4249, 1519, 9829]])

In [47]:
faiss.write_index(indexIVFPQ, "vaswani_faiss_ivfpq.ndx")

### Tarea 
**Ejecutar todo el set de consultas y calcular el solapamiento promedio de las listas de 
resultados para nprobe = [1 .. 10]**